In [1]:
import pandas as pd
from xbbg import blp
import matplotlib.pyplot as plt

import string


import sys
sys.path.insert(0, '../cmds')
from utils import *

# Parameters

In [2]:
SAVE_DATA = True

STARTDATE = '2018-01-01'
ENDDATE = '2024-01-15'

# SOFR Swaps

In [3]:
DO_MONTHS = True

month_codes = string.ascii_uppercase[0:11]
tenors = [1,2,3,4,5,6,7,8,9,10,15,20,25,30,40,50]

ticks = list()

for code in tenors:
    ticks.append(f'USOSFR{code} Curncy')


if DO_MONTHS:
    for code in month_codes:
        ticks.append(f'USOSFR{code} Curncy')

    for code in month_codes:
        ticks.append(f'USOSFR1{code} Curncy')

    for code in month_codes[2::3]:
        ticks.append(f'USOSFR2{code} Curncy')    
    

maturities = tenors
if DO_MONTHS:
    maturities += [i/12 for i in range(1,12)]
    maturities += [1+i/12 for i in range(1,12)]
    maturities += [2+i/12 for i in range(3,12,3)]
        
map_tick_des = dict(keys=maturities,values=ticks)

In [4]:
swaps = blp.bdh(tickers=ticks, flds='last_price',\
        start_date=STARTDATE, end_date=ENDDATE).droplevel(level=1,axis=1)

swaps.index = pd.to_datetime(swaps.index)

swaps.columns = maturities
swaps.sort_index(axis=1,inplace=True)

In [5]:
info = pd.DataFrame(map_tick_des)
info.columns = ['maturity','BB ticker']
info = info.set_index('maturity').sort_index()

In [ ]:
if SAVE_DATA:
    outfile = f'../data/sofr_swaps.xlsx'
    with pd.ExcelWriter(outfile) as writer:  
        info.to_excel(writer, sheet_name= 'info', index=True)
        swaps.to_excel(writer, sheet_name= 'data', index=True)

## References

### Why the spike in SOFR on Sep 17, 2019?
https://www.federalreserve.gov/econres/notes/feds-notes/what-happened-in-money-markets-in-september-2019-20200227.htm

https://en.wikipedia.org/wiki/September_2019_events_in_the_U.S._repo_market